|Trading Indictators|
|-|

In [3]:
#libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

#libs -> data collection
import requests
from datetime import datetime

#keys
from config import IEX_KEY


In [6]:
# Replace 'your_api_key' with your actual IEX Cloud API key
api_key = IEX_KEY
symbol = 'NOW' # servicenow

# Define the base URL and endpoint for the IEX Cloud API
base_url = 'https://cloud.iexapis.com/stable'
endpoint = f'/stock/{symbol}/quote'

# Set up the request URL with your API key
url = f'{base_url}{endpoint}?token={api_key}'

# Send an HTTP GET request to the IEX Cloud API and retrieve the response
response = requests.get(url)
data = response.json()

# Print the stock data for TSLA
print(f"Symbol: {data['symbol']}")
print(f"Company Name: {data['companyName']}")
print(f"Latest Price: ${data['latestPrice']}")
print(f"Previous Close: ${data['previousClose']}")
print(f"Change: ${data['change']} ({data['changePercent']}%)")

Symbol: NOW
Company Name: ServiceNow Inc
Latest Price: $475.59
Previous Close: $475.99
Change: $-0.4 (-0.00084%)
